**Machine Learning for Time Series (Master MVA)**

- [Link to the class material.](http://www.laurentoudre.fr/ast.html)

# Introduction

In this tutorial, we illustrate the following concepts:

- outlier detection/removal,
- matrix profile.

In [ ]:
import datetime as dt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from loadmydata.load_nyc_taxi import load_nyc_taxi_dataset
from numpy.fft import rfft, rfftfreq
from numpy.polynomial.polynomial import Polynomial
from scipy.signal import butter, sosfilt
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import StandardScaler

In [ ]:
try:
    from numpy.lib.stride_tricks import sliding_window_view  # New in version 1.20.0

    def get_trajectory_matrix(arr, window_shape, jump=1):
        return sliding_window_view(x=arr, window_shape=window_shape)[::jump]

except ImportError:

    def get_trajectory_matrix(arr, window_shape, jump=1):
        n_rows = ((arr.size - window_shape) // jump) + 1
        n = arr.strides[0]
        return np.lib.stride_tricks.as_strided(
            arr, shape=(n_rows, window_shape), strides=(jump * n, n)
        )

In [ ]:
def fig_ax(figsize=(15, 5)):
    return plt.subplots(figsize=figsize)

In [ ]:
def fill_band(array2D: np.ndarray, width: int = 1, value=0) -> np.ndarray:
    """Fill thick diagonal band of a matrix with value"""
    n_rows, n_cols = array2D.shape
    distance_from_diag = np.abs(np.add.outer(np.arange(n_rows), -np.arange(n_cols)))
    array2D[distance_from_diag <= width] = value
    return array2D

## Outliers detection/removal

In [ ]:
X, _, description = load_nyc_taxi_dataset()

print(description)

In [ ]:
original_calendar_time_array = X.timestamp.to_numpy()
original_taxi_count_np = X.taxi_count.to_numpy()

In [ ]:
daily_taxi_count = X.resample("1D", on="timestamp").sum()
daily_taxi_count_np = daily_taxi_count.to_numpy().squeeze()
calendar_time_array = daily_taxi_count.index.to_numpy()
n_samples = daily_taxi_count_np.size
fig, ax = fig_ax()
ax.plot(daily_taxi_count, "*-")
_ = ax.set_ylim(0)

### Distribution

On the original data.

In [ ]:
quantile_threshold_low, quantile_threshold_high = 0.01, 0.997

fig, ax = fig_ax()
_ = ax.hist(daily_taxi_count_np, 20)

threshold_low, threshold_high = np.quantile(
    daily_taxi_count_np, [quantile_threshold_low, quantile_threshold_high]
)

_ = ax.axvline(threshold_low, ls="--", color="k")
_ = ax.axvline(threshold_high, ls="--", color="k")

<div class="alert alert-success" role="alert">
    <p><b>Question</b></p>
    <p>In the previous cell, modify <tt>quantile_threshold_low</tt> and <tt>quantile_threshold_high</tt> to only exclude outliers.</p>
</div>

Plot the outliers directly on the signal.

In [ ]:
fig, ax = fig_ax()
ax.plot(calendar_time_array, daily_taxi_count_np, "*-", label="Daily taxi count")

outlier_mask = (daily_taxi_count_np < threshold_low) | (
    daily_taxi_count_np > threshold_high
)

ax.plot(
    calendar_time_array[outlier_mask],
    daily_taxi_count_np[outlier_mask],
    "*",
    label="Outliers",
)

plt.legend()
_ = ax.set_ylim(0)

<div class="alert alert-success" role="alert">
    <p><b>Question</b></p>
    <p>Repeat the same procedure on the distribution of the previous seasonal fit.</p>
</div>

This method can be extended for any type of signal approximation (SSA, polynomial, smoothing, etc.)

### Matrix profile

Informally, an outlier is a motif that only appears once a signal.
The distance of this particular pattern from all other patterns is large.

Algorithmically:

- Extract the trajectory matrix.
- Compute the pairwise distances between all patterns.
- Set to Inf the distance between patterns that overlap.
- Take the minimum pairwise distance for each pattern (the profile)


Then we apply outlier detection on the profile.

In [ ]:
window_shape = 7
quantile_threshold_high = 0.9

In [ ]:
# extract the trajectory matrix
trajectory_matrix = get_trajectory_matrix(
    arr=daily_taxi_count_np, window_shape=window_shape
)

# compute distance matrix
distance_matrix = squareform(pdist(trajectory_matrix, metric="correlation"))
plt.imshow(distance_matrix)

In [ ]:
# set to inf the overlapping windows
distance_matrix = fill_band(distance_matrix, window_shape, np.inf)
plt.imshow(distance_matrix)

In [ ]:
# compute profile
profile = distance_matrix.min(axis=1)

We can now do outlier detection on the profile

In [ ]:
# get threshold
threshold_high = np.quantile(profile, quantile_threshold_high)


# plot results
fig, (ax_0, ax_1) = plt.subplots(
    1, 2, gridspec_kw={"width_ratios": [1, 2]}, figsize=(20, 5)
)

ax_0.plot(profile)
ax_0.axhline(threshold_high, ls="--", color="k")

ax_1.plot(calendar_time_array, daily_taxi_count_np, "*-", label="Daily taxi count")
outlier_mask = profile > threshold_high
offset = window_shape - 1
ax_1.plot(
    calendar_time_array[:-offset][outlier_mask],
    daily_taxi_count_np[:-offset][outlier_mask],
    "*",
    label="Outliers",
)
_ = plt.legend()

<div class="alert alert-success" role="alert">
    <p><b>Question</b></p>
    <p>Repeat the same experiment on the original (not daily) signal for windows of one day, two days and a week. 
    Report the dates that you find interesting.</p>
</div>